In [64]:
# ! pip uninstall transformers -y
# ! pip install transformers==4.35.2
! pip install ml-dtypes==0.2.0




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.3
    Uninstalling ml_dtypes-0.5.3:
      Successfully uninstalled ml_dtypes-0.5.3


In [5]:
pip show transformers tensorflow keras ml-dtypes onnx torch


Name: transformers
Version: 4.35.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /raid/invigoworks/anaconda3/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: sentence-transformers
---
Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /raid/invigoworks/anaconda3/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dty

Author: 
Author-email: ONNX Contributors <onnx-technical-discuss@lists.lfaidata.foundation>
License: Apache License v2.0
Location: /raid/invigoworks/anaconda3/lib/python3.10/site-packages
Requires: ml_dtypes, numpy, protobuf, typing_extensions
Required-by: onnxmltools, skl2onnx
---
Name: torch
Version: 2.3.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /raid/invigoworks/anaconda3/lib/python3.10/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: sentence-transformers, torchaudio, torchvision
Note: you may need to restart the kernel to use updated pac

In [ ]:
import transformers
from transformers import pipeline
print(pipeline)
print(transformers.__file__)

In [5]:
# Standard Library Imports
import os
import sys
import types
import time
import random
import glob
import re
import requests
from datetime import datetime, timedelta

# Data Handling and Scientific Computing
import numpy as np
import pandas as pd
import warnings

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Financial Data
import yfinance as yf

# Machine Learning - Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix
)

# Deep Learning - TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Deep Learning - PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Natural Language Processing (Hugging Face Transformers)
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification

# Google Trends
from pytrends.request import TrendReq
from time import sleep
warnings.filterwarnings('ignore')

# 설정
ROOT_DIR = "./news_data"   
FIN_NAME = "snunlp/KR-FinBert-SC"
START_DATE = "2020-01-01"
END_DATE = "2025-10-01"
TRAIN_END = "2024-12-31"
TICKER = "ETH-USD"
SEQ_LEN = 3
ETF_APPROVAL_DATE = "2024-01-10"  # 이더리움 ETF 승인 날짜
np.random.seed(42); tf.random.set_seed(42)

In [6]:
def parse_date_from_filename(filename):
    patterns = [
        r'(\d{4})-(\d{2})-(\d{2})',
        r'(\d{4})(\d{2})(\d{2})',
        r'(\d{2})-(\d{2})-(\d{4})',
        r'(\d{2})(\d{2})(\d{4})'
    ]
    basename = os.path.basename(filename)
    for pattern in patterns:
        match = re.search(pattern, basename)
        if match:
            try:
                if len(match.group(1)) == 4:
                    year, month, day = match.groups()
                else:
                    day, month, year = match.groups()
                return pd.to_datetime(f"{year}-{month}-{day}")
            except:
                continue
    return None

In [7]:
def load_all_news_data(root_dir):
    """2020.01.01부터 현재까지의 모든 뉴스 CSV 파일 로드"""
    all_data = []
    
    if not os.path.exists(root_dir):
        raise FileNotFoundError(f"디렉토리가 존재하지 않습니다: {root_dir}")
    
    csv_files = sorted([f for f in os.listdir(root_dir) if f.endswith('.csv')])
    
    for filename in csv_files:
        
        filepath = os.path.join(root_dir, filename)
        file_date=parse_date_from_filename(filename)
        
        for enc in ['utf-8', 'cp949', 'latin1']:
            try:
                df = pd.read_csv(filepath, encoding=enc)
                break
            except UnicodeDecodeError:
                continue
        else:
            print(f"읽기 실패: {filepath}")
            continue
        
        df=df.loc[:,['news','label'] if 'date' not in df.columns else ['news','label','date']]
        if 'date' not in df.columns:
            df['date']=file_date
        else:
            df['date']=pd.to_datetime(df['date'],errors='coerce')
            if file_date is not None:
                df['date'] = df['date'].fillna(file_date)
            
        all_data.append(df)

        
    
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df['date'] = pd.to_datetime(combined_df['date'], errors='coerce')

    
    return combined_df

# 뉴스 데이터 로드
news_df = load_all_news_data(ROOT_DIR)
print(f"총 뉴스: {len(news_df)}건, 기간: {news_df['date'].min()} ~ {news_df['date'].max()}")


총 뉴스: 25910건, 기간: 2020-01-01 00:00:00 ~ 2025-10-01 00:00:00


In [9]:
# news_df.loc[news_df['date'].isnull(),'date']=pd.to_datetime('2025-09-23')

# target_date=pd.to_datetime('2025-09-23')
# mask=news_df['date']==target_date
# news_df.loc[mask,'news']=''
# print(news_df.isnull().sum())

print(news_df.isnull().sum())

news     0
label    0
date     0
dtype: int64


In [ ]:
# ========================================
# 2. FinBERT 감성분석
# ========================================


def analyze_sentiment_finbert(texts, model_name='ProsusAI/finbert', batch_size=32):
    """
    FinBERT로 뉴스 감성분석
    
    Parameters:
    - texts: 뉴스 텍스트 리스트
    - model_name: 'ProsusAI/finbert' 또는 'yiyanghkust/finbert-tone'
    
    Returns:
    - sentiment_scores: 감성 점수 (-1 ~ 1, positive - negative)
    """
    print(f"FinBERT 감성분석 시작: {model_name}")
    
    # 모델 및 토크나이저 로드
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    
    print(f"디바이스: {device}")
    
    sentiment_scores = []
    
    # 배치 처리
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        
        # 토큰화
        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # 예측
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            probs = probs.cpu().numpy()
        
        # 감성 점수 계산
        if model_name == 'ProsusAI/finbert':
            # [positive, negative, neutral]
            batch_scores = probs[:, 0] - probs[:, 1]
        else:  # yiyanghkust/finbert-tone
            # [neutral, positive, negative]
            batch_scores = probs[:, 1] - probs[:, 2]
        
        sentiment_scores.extend(batch_scores.tolist())
        
        if (i // batch_size + 1) % 20 == 0:
            print(f"진행: {i+len(batch_texts)}/{len(texts)}")
    
    return sentiment_scores

# 감성 점수 계산
news_df['sentiment_score'] = analyze_sentiment_finbert(
    news_df['news'].fillna('').tolist(),
    model_name='ProsusAI/finbert'
)

# 일별 감성 점수 집계
daily_sentiment = news_df.groupby('date').agg({
    'sentiment_score': ['mean', 'std', 'max', 'min', 'count']
}).reset_index()

daily_sentiment.columns = ['Date', 'sentiment_mean', 'sentiment_std', 
                           'sentiment_max', 'sentiment_min', 'news_count']
daily_sentiment['sentiment_std'] = daily_sentiment['sentiment_std'].fillna(0)

print(f"일별 감성 데이터: {len(daily_sentiment)}일")

def analyze_sentiment_finbert_single(text, model_name='ProsusAI/finbert'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        probs = probs.cpu().numpy()
    if model_name == 'ProsusAI/finbert':
        score = probs[0, 0] - probs[0, 1]
    else:
        score = probs[0, 1] - probs[0, 2]
    return score

# 하루 단위로 뉴스 합치기
daily_all_news = news_df.groupby('date')['news'].apply(lambda x: ' '.join(x.dropna()))

# 감성 분석
daily_sent_score = daily_all_news.apply(lambda x: analyze_sentiment_finbert_single(x, 'ProsusAI/finbert'))
daily_sent_score = pd.DataFrame({'date': daily_all_news.index, 'sentiment_score_single': daily_sent_score.values})

print(daily_sent_score.head())

FinBERT 감성분석 시작: ProsusAI/finbert
디바이스: cuda
진행: 640/25910
진행: 1280/25910
진행: 1920/25910
진행: 2560/25910
진행: 3200/25910
진행: 3840/25910
진행: 4480/25910
진행: 5120/25910
진행: 5760/25910
진행: 6400/25910
진행: 7040/25910
진행: 7680/25910
진행: 8320/25910
진행: 8960/25910
진행: 9600/25910
진행: 10240/25910
진행: 10880/25910
진행: 11520/25910
진행: 12160/25910
진행: 12800/25910
진행: 13440/25910
진행: 14080/25910
진행: 14720/25910
진행: 15360/25910
진행: 16000/25910
진행: 16640/25910
진행: 17280/25910
진행: 17920/25910
진행: 18560/25910
진행: 19200/25910
진행: 19840/25910
진행: 20480/25910
진행: 21120/25910
진행: 21760/25910
진행: 22400/25910
진행: 23040/25910
진행: 23680/25910
진행: 24320/25910
진행: 24960/25910
진행: 25600/25910
일별 감성 데이터: 2059일


In [ ]:
print(daily_sentiment.tail())

In [67]:
# ========================================
# 3. 이더리움 가격 데이터 수집
# ========================================
import yfinance as yf

# 이더리움 가격 다운로드
print("이더리움(ETH-USD) 가격 다운로드...")
eth_data = yf.download(TICKER, START_DATE, END_DATE, progress=False)


eth_data = eth_data.reset_index()  
eth_data.columns = [col[0] if isinstance(col, tuple) else col for col in eth_data.columns]

# 가격 변동 및 타겟 생성
eth_data['Price_Change'] = eth_data['Close'].diff()
eth_data['Price_Change_Pct'] = eth_data['Close'].pct_change() * 100

# 다음날 방향 (1=상승, 0=하락)
eth_data['Next_Day_Direction'] = (eth_data['Close'].shift(-1) > eth_data['Close']).astype(int)

eth_data['Date'] = pd.to_datetime(eth_data['Date'])

print(f"이더리움 데이터: {len(eth_data)}일, 기간: {eth_data['Date'].min()} ~ {eth_data['Date'].max()}")

print(eth_data.head())

이더리움(ETH-USD) 가격 다운로드...
이더리움 데이터: 2100일, 기간: 2020-01-01 00:00:00 ~ 2025-09-30 00:00:00
        Date       Close        High         Low        Open       Volume  \
0 2020-01-01  130.802002  132.835358  129.198288  129.630661   7935230330   
1 2020-01-02  127.410179  130.820038  126.954910  130.820038   8032709256   
2 2020-01-03  134.171707  134.554016  126.490021  127.411263  10476845358   
3 2020-01-04  135.069366  136.052719  133.040558  134.168518   7430904515   
4 2020-01-05  136.276779  139.410202  135.045624  135.072098   7526675353   

   Price_Change  Price_Change_Pct  Next_Day_Direction  
0           NaN               NaN                   0  
1     -3.391823         -2.593097                   1  
2      6.761528          5.306898                   1  
3      0.897659          0.669038                   1  
4      1.207413          0.893920                   1  


In [34]:
# ========================================
# 4. 데이터 병합 및 특징 생성
# ========================================

merged_data = pd.merge(eth_data, daily_sentiment, on='Date', how='left')
# 가격 데이터와 감성 데이터 병합


# 결측치 처리
merged_data['sentiment_mean'] = merged_data['sentiment_mean'].fillna(0)
merged_data['sentiment_std'] = merged_data['sentiment_std'].fillna(0)
merged_data['sentiment_max'] = merged_data['sentiment_max'].fillna(0)
merged_data['sentiment_min'] = merged_data['sentiment_min'].fillna(0)
merged_data['news_count'] = merged_data['news_count'].fillna(0)

# 이동평균 특징 추가
merged_data['MA_7'] = merged_data['Close'].rolling(window=7).mean()
merged_data['MA_30'] = merged_data['Close'].rolling(window=30).mean()
merged_data['Volatility_7'] = merged_data['Close'].rolling(window=7).std()

# 감성 이동평균
merged_data['Sentiment_MA_3'] = merged_data['sentiment_mean'].rolling(window=3).mean()
merged_data['Sentiment_MA_7'] = merged_data['sentiment_mean'].rolling(window=7).mean()

# NaN 제거
merged_data = merged_data.dropna()

print(f"최종 데이터: {len(merged_data)}일")
print(f"타겟 분포: {merged_data['Next_Day_Direction'].value_counts().to_dict()}")
print(merged_data.head())

최종 데이터: 2071일
타겟 분포: {1: 1080, 0: 991}
         Date       Date       Close        High         Low        Open  \
29 2020-01-30 2020-01-30  184.690475  186.260483  172.374634  174.917709   
30 2020-01-31 2020-01-31  180.160172  185.405838  176.296814  184.736908   
31 2020-02-01 2020-02-01  183.673950  183.845551  179.745178  180.113770   
32 2020-02-02 2020-02-02  188.617538  193.080399  180.173950  183.532501   
33 2020-02-03 2020-02-03  189.865067  193.436890  188.012695  188.607407   

         Volume  Price_Change  Price_Change_Pct  Next_Day_Direction  \
29  12604789338      9.640137          5.507065                   0   
30  11728616394     -4.530304         -2.452917                   1   
31  11569697182      3.513779          1.950364                   1   
32  14054425389      4.943588          2.691502                   1   
33  12392875241      1.247528          0.661406                   0   

    sentiment_mean  sentiment_std  sentiment_max  sentiment_min  news_count  

In [24]:
# ========================================
# 5. BiLSTM 모델 구현
# ========================================

class TimeSeriesDataset(Dataset):
    """시계열 데이터셋"""
    def __init__(self, sequences, labels):
        self.sequences = torch.FloatTensor(sequences)
        self.labels = torch.LongTensor(labels)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

class FinBERT_BiLSTM(nn.Module):
    """FinBERT 감성 점수 + BiLSTM 모델"""
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.3):
        super(FinBERT_BiLSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # BiLSTM 레이어
        self.bilstm = nn.LSTM(
            input_dim, 
            hidden_dim, 
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Attention 레이어
        self.attention = nn.Linear(hidden_dim * 2, 1)
        
        # 분류 레이어
        self.fc1 = nn.Linear(hidden_dim * 2, 64)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # 이진 분류 (상승/하락)
        
        self.relu = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.batch_norm2 = nn.BatchNorm1d(32)
    
    def forward(self, x):
        # BiLSTM
        lstm_out, _ = self.bilstm(x)
        
        # Attention mechanism
        attention_weights = torch.softmax(self.attention(lstm_out), dim=1)
        context_vector = torch.sum(attention_weights * lstm_out, dim=1)
        
        # 분류
        out = self.fc1(context_vector)
        out = self.batch_norm1(out)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        out = self.batch_norm2(out)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc3(out)
        
        return out


In [25]:
# ========================================
# 6. 시퀀스 데이터 생성
# ========================================

def create_sequences(data, feature_cols, target_col, seq_length=10):
    X, y = [], []
    
    for i in range(len(data) - seq_length):
        X.append(data[feature_cols].iloc[i:i+seq_length].values)
        y.append(data[target_col].iloc[i+seq_length])
    
    return np.array(X), np.array(y)

feature_cols = ['Open', 'High', 'Low', 'Close', 'Volume',
                'sentiment_mean', 'sentiment_std', 'sentiment_max', 
                'sentiment_min', 'news_count',
                'MA_7', 'MA_30', 'Volatility_7',
                'Sentiment_MA_3', 'Sentiment_MA_7']

target_col = 'Next_Day_Direction'
seq_length = 10

X, y = create_sequences(merged_data, feature_cols, target_col, seq_length)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15, shuffle=False)

scaler = StandardScaler()
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)

X_val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_val_scaled = scaler.transform(X_val_reshaped).reshape(X_val.shape)

X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])
X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape)

model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(seq_length, len(feature_cols))),
    Dropout(0.3),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(32)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)

history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Down', 'Up']))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Epoch 1/100
47/47 [==============================] - 9s 39ms/step - loss: 0.6939 - accuracy: 0.4966 - val_loss: 0.6945 - val_accuracy: 0.4867 - lr: 0.0010
Epoch 2/100
47/47 [==============================] - 0s 10ms/step - loss: 0.6934 - accuracy: 0.5067 - val_loss: 0.6937 - val_accuracy: 0.4829 - lr: 0.0010
Epoch 3/100
47/47 [==============================] - 0s 10ms/step - loss: 0.6905 - accuracy: 0.5289 - val_loss: 0.6936 - val_accuracy: 0.4867 - lr: 0.0010
Epoch 4/100
47/47 [==============================] - 0s 10ms/step - loss: 0.6934 - accuracy: 0.5208 - val_loss: 0.6934 - val_accuracy: 0.4867 - lr: 0.0010
Epoch 5/100
47/47 [==============================] - 0s 10ms/step - loss: 0.6913 - accuracy: 0.5370 - val_loss: 0.6956 - val_accuracy: 0.5095 - lr: 0.0010
Epoch 6/100
47/47 [==============================] - 0s 10ms/step - loss: 0.6890 - accuracy: 0.5410 - val_loss: 0.6964 - val_accuracy: 0.5095 - lr: 0.0010
Epoch 7/100
47/47 [==============================] - 0s 10ms/step - lo